In [1]:
#Add Dependencies
import pandas as pd
import numpy as np
# import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
read_file = "../Working/movies_metadata.csv"
write_file = "../Resources/movies_clean.csv"

#read_file = "/Users/archieanand/Desktop/Analysis/Working/movies_metadata.csv"
#write_file = "/Users/archieanand/Desktop/Analysis/Working/movies_clean.csv"

In [3]:
# Import our input dataset
movies_df = pd.read_csv(read_file , low_memory=False, index_col="imdb_id")

In [4]:
#Check imported Dataset
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0114709,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
tt0113497,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
tt0113228,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
tt0114885,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
tt0113041,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
#Check Dataset Column Types
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45466 entries, tt0114709 to tt6980792
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   original_language      45455 non-null  object 
 7   original_title         45466 non-null  object 
 8   overview               44512 non-null  object 
 9   popularity             45461 non-null  object 
 10  poster_path            45080 non-null  object 
 11  production_companies   45463 non-null  object 
 12  production_countries   45463 non-null  object 
 13  release_date           45379 non-null  object 
 14  revenue                45460 non-null  float64


In [6]:
#Drop movies considered "Adult"
movies_df = movies_df.drop(movies_df[movies_df.adult == "True"].index)

In [7]:
#Drop movies that are not Released (Only Released movies would be considered for Oscars)
movies_df = movies_df.drop(movies_df[movies_df.status != "Released"].index)

In [8]:
#Drop columns not needed for Evaluation
movies_df= movies_df.drop(columns=["adult", "status", "homepage", "belongs_to_collection", "id","original_language", "overview", "poster_path", "spoken_languages", "tagline", "video"    ])
movies_df.head()

,budget,genres,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count
imdb_id,,,,,,,,,,,,
tt0114709,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0
tt0113497,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0
tt0113228,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0
tt0114885,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0
tt0113041,0,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0


In [9]:
#Process Production Companies Column and set as first company in list
for index, row in movies_df.iterrows():    
    try:        
        companies = movies_df.loc[index, 'production_companies']        
        company = eval(companies)[0]['name']    
    except:        
        company = "Other"    
    movies_df.at[index, 'production'] = company

In [10]:
#Process Production Countries Column and set as first Country in list
for index, row in movies_df.iterrows():    
    try:        
        countries = movies_df.loc[index, 'production_countries']        
        country = eval(countries)[0]['name']    
    except:        
        country = "Other"    
    movies_df.at[index, 'country'] = country

In [11]:
#Process Genres Column and set as first genre in list
for index, row in movies_df.iterrows():    
    try:        
        genres = movies_df.loc[index, 'genres']        
        genre = eval(genres)[0]['name']    
    except:        
        genres = "Other"    
    movies_df.at[index, 'new_genre'] = genre

In [12]:
#Convert Release Date to contain just the year (Exact date is unnecessary)
movies_df['Release_year'] = movies_df['release_date'].str[:4]

In [13]:
#Check remaining columns
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44991 entries, tt0114709 to tt6980792
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                44991 non-null  object 
 1   genres                44991 non-null  object 
 2   original_title        44991 non-null  object 
 3   popularity            44991 non-null  object 
 4   production_companies  44991 non-null  object 
 5   production_countries  44991 non-null  object 
 6   release_date          44916 non-null  object 
 7   revenue               44991 non-null  float64
 8   runtime               44742 non-null  float64
 9   title                 44991 non-null  object 
 10  vote_average          44991 non-null  float64
 11  vote_count            44991 non-null  float64
 12  production            44991 non-null  object 
 13  country               44991 non-null  object 
 14  new_genre             44991 non-null  object 
 15  Release_year

In [14]:
#Verify Current Dataset
movies_df

,budget,genres,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,Release_year
imdb_id,,,,,,,,,,,,,,,,
tt0114709,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995
tt0113497,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,1995
tt0113228,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,1995
tt0114885,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,United States of America,Comedy,1995
tt0113041,0,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,Sandollar Productions,United States of America,Comedy,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt6209470,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",رگ خواب,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,Subdue,4.0,1.0,Other,Iran,Drama,NaN
tt2028550,0,"[{'id': 18, 'name': 'Drama'}]",Siglo ng Pagluluwal,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,Century of Birthing,9.0,3.0,Sine Olivia,Philippines,Drama,2011
tt0303758,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",Betrayal,0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,Betrayal,3.8,6.0,American World Pictures,United States of America,Action,2003


In [15]:
#Drop processed columns, keeping only clean dataset
movies_df= movies_df.drop(columns=["genres", "production_companies", "production_countries", "release_date"])
movies_df.head()

,budget,original_title,popularity,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,Release_year
imdb_id,,,,,,,,,,,,
tt0114709,30000000,Toy Story,21.946943,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995
tt0113497,65000000,Jumanji,17.015539,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,1995
tt0113228,0,Grumpier Old Men,11.7129,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,1995
tt0114885,16000000,Waiting to Exhale,3.859495,81452156.0,127.0,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,United States of America,Comedy,1995
tt0113041,0,Father of the Bride Part II,8.387519,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,Sandollar Productions,United States of America,Comedy,1995


In [17]:
#Set index to IMDB ID
#movies_df.set_index("imdb_id")

In [18]:
#Convert Budget and Popularty colums from Object to Numbers
movies_df['budget']=movies_df.budget.astype('int64')
movies_df['popularity']=movies_df.budget.astype('float64')

In [19]:
#Drop any rows containing blank data
movies_df = movies_df.dropna(axis = 0,how= 'any')

In [20]:
#Check final dataset numbers
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44678 entries, tt0114709 to tt6980792
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   budget          44678 non-null  int64  
 1   original_title  44678 non-null  object 
 2   popularity      44678 non-null  float64
 3   revenue         44678 non-null  float64
 4   runtime         44678 non-null  float64
 5   title           44678 non-null  object 
 6   vote_average    44678 non-null  float64
 7   vote_count      44678 non-null  float64
 8   production      44678 non-null  object 
 9   country         44678 non-null  object 
 10  new_genre       44678 non-null  object 
 11  Release_year    44678 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 4.4+ MB


In [22]:
#Export dataset to CSV
movies_df.to_csv(write_file)